In [1]:
import glob
import os
from utils import *

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from plot_settings import *

from scipy.optimize import curve_fit
from scipy.interpolate import interp1d, UnivariateSpline
from collections import Counter

Inputs the combined output manual vcheck files and calculates rates, radii and other descriptors for the pH profiles and saves a combined dataframe for all the profiles

In [2]:
# Important parameters

reference_pH = 8.5      # used for the calculation of radius and rates
min_radius = 3.4        # minimum droplet size used in the analysis
target_pH = 7.5         # droplets which don't form vacuoles are tracked till this pH

In [3]:
experiments = ['23.11.14-15', '23.12.22-23', '24.04.01', '24.07.08', '24.11.09', '24.11.14']
# experiments = ['24.11.09', '24.11.14']
# experiments = ['23.11.14-15', '23.12.22-23', '24.04.01', '24.07.08']
asymptotic_exp = ['24.11.09', '24.11.14']
# experiments = ['24.11.09', '24.11.14']


# Curvature Parameter - How much curvature is too much curvature?

- we fit pH profiles from pH=8.7 till the point of vacuole formation with a parabolic fit

- the fittings are saved in the folder fitting_v7

- the curvature parameter is defined as

$$C = \frac{\frac{d^2pH}{dt^2}(\Delta pH)}{\bigg(\frac{dpH}{dt}\bigg)^2}$$

where we choose $\Delta pH = 1$ 
if $C \sim 0 =>$ straight line  
if $C >> 0 =>$ curving down  
if $C << 0 =>$ curving up  

Tells you how much does the profile deviates from a straight line

# pH uncertainity

pH has asymmetric uncertainity in the data. we need symmetric uncertainity for error propagation while calculating the rate parameter. taking the geometric mean of the asymmeytric uncertainity to get the symmetric uncertainity 

$$\sigma^2 = (\sigma_{+})(\sigma_{-})$$

# Parabolic fitting and Error Propagation in curve fitting

## Option 1

fit pH to 
$$pH(t) = a t^2 + bt +c$$

then calculate the slope at $t=t_0$, where $t_0$ is the time when pH is same as the reference pH

$$\frac{dpH}{dt} = 2at_0 + b$$

### Error Propagation in scalar function

$$f = \sum_i a_ix_i = \mathbf{a}\mathbf{x}$$

$$\sigma_f^2 = \mathbf{a} \Sigma \mathbf{a}^T $$

if $\Sigma$ is uncorrelated $\sigma_f^2 =  \sum_i \sigma_i^2 a_i^2 $


## Option 2

fit pH to 
$$pH(t) = pH_0 + \frac{dpH}{dt}(t-t_0) + \frac{1}{2}\frac{d^2pH}{dt^2}(t-t_0)^2$$

$$pH(t) = \frac{1}{2}\frac{d^2pH}{dt^2} t^2 + \bigg[ \frac{dpH}{dt} - \frac{d^2pH}{dt^2}t_0\bigg]t + pH_0 -\frac{dpH}{dt}t_0 +\frac{1}{2}\frac{d^2pH}{dt^2}t_0^2$$

$$\frac{dpH}{dt} = 2at_0 + b$$

### Error Propagation in scalar function

$$f = \sum_i a_ix_i = \mathbf{a}\mathbf{x}$$

$$\sigma_f^2 = \mathbf{a} \Sigma \mathbf{a}^T $$

if $\Sigma$ is uncorrelated $\sigma_f^2 =  \sum_i \sigma_i^2 a_i^2 $

In [4]:
combined_df = load_experimental_data([f'combined_output_{exp}_manual_vcheck.csv' for exp in experiments])
combined_df = compute_ph_sym_uncertainty(combined_df)

# Initialize results dictionary

pH_fit_range_non_vacuole= [7.5, reference_pH+0.2]
pH_fit_range = [reference_pH-0.2, reference_pH+0.2]

drop_param = {
    'particle': [],             # all
    'vacuole': [],              # all
    'radius': [],               # radius at ref pH
    't0': [],                   # time at ref pH  
    'tf': [],                   # all
    'min_pH': [],               # all
    'initial_pH': [],           # all
    'pH_vacuole': [],           # all
    'tv': [],                   # conditioned
    'pH_vacuole_sigmap': [],    # uncertainity in pH for vacuole formation sigma+; conditioned
    'pH_vacuole_sigmam': [],    # uncertainity in pH for vacuole formation sigma-; conditioned
    'rate(dpHdt)': [],          # conditioned
    'rate_sigma': [],           # uncertainity in rate; conditioned
    'd2pHdt2': [],              # conditioned
    'curvature_param': [],      # conditioned        
    'consider_drop': [],        # all
    'experiment': [],           # all
    'trackID': []               # all
}

# Process each particle
for particle_ID in combined_df['particle'].unique():
    data = combined_df[combined_df['particle'] == particle_ID]

    drop_param['particle'].append(particle_ID)
    drop_param['vacuole'].append(data['vacuole'].iloc[0])

    r0, t0 = get_ref_radius_and_time(data, reference_pH=reference_pH)
    drop_param['radius'].append(r0)
    drop_param['t0'].append(t0)
    drop_param['tf'].append(data['time(sec)'].iloc[-1])
    drop_param['min_pH'].append(data['pH'].min())
    drop_param['initial_pH'].append(data['pH'].iloc[0])

    pH_v = data['pH_vfirst'].iloc[0]
    drop_param['pH_vacuole'].append(pH_v)
    tv = get_time_for_vacuole_formation(data, pH_v)
    sigmap, sigmam = get_uncertainity_in_pHv(data, pH_v)
    drop_param['tv'].append(tv)
    drop_param['pH_vacuole_sigmap'].append(sigmap)
    drop_param['pH_vacuole_sigmam'].append(sigmam)

    reaches_target_pH = int(drop_param['min_pH'][-1] <= target_pH)
    if drop_param['initial_pH'][-1] < reference_pH:
        consider_drop = 0
    elif drop_param['vacuole'][-1] == 0 and not reaches_target_pH:
        consider_drop = 0
    elif drop_param['vacuole'][-1] == 0.5:
        consider_drop = 0
    else:
        consider_drop = 1

    drop_param['experiment'].append(data['exp_date'].iloc[0])
    drop_param['trackID'].append(data['trackID'].iloc[0])

    if drop_param['experiment'][-1] in asymptotic_exp:  # consider all droplets for asymptotic experiment
        consider_drop = 1
    drop_param['consider_drop'].append(consider_drop)
    
    initial_guess= [-0.00001,-0.001,8.] 
    tolerance = 1e-7

    if drop_param['experiment'][-1] in asymptotic_exp:
        popt, perr = fit_ph_rate(data, drop_param['t0'][-1], pH_fit_range, initial_guess, tolerance)
    elif consider_drop == 1 and drop_param['vacuole'][-1] == 0:        
        popt, perr = fit_ph_rate(data, drop_param['t0'][-1], pH_fit_range_non_vacuole, initial_guess, tolerance)
    elif consider_drop == 1 and drop_param['vacuole'][-1] == 1:
        popt, perr = fit_ph_rate(data, drop_param['t0'][-1], [drop_param['pH_vacuole'][-1], reference_pH+0.2], initial_guess, tolerance)
    else:
        popt = [np.nan, np.nan, np.nan]
        perr = [np.nan, np.nan, np.nan]

    a,b,c = popt
    drop_param['rate(dpHdt)'].append(b)
    drop_param['d2pHdt2'].append(2 * a)
    drop_param['rate_sigma'].append(perr[1])
    drop_param['curvature_param'].append(2 * a / b**2 if b != 0 else np.nan)
    
    # Plot fitting
    exp_date = data['exp_date'].values[0]
    f_location = data['f_location'].values[0]
    drop = data['particleID_original'].values[0]
    plot_name = f"{exp_date} - f{f_location} - drop{drop} - vacuole{drop_param['vacuole'][-1]} - pH_v {drop_param['pH_vacuole'][-1]} - dropn{drop_param['particle'][-1]}.png" 
    plot_fitting(data, popt, drop_param['t0'][-1], drop_param['pH_vacuole'][-1], pH_fit_range_non_vacuole, f'fitting_v7_test/{data["exp_date"].values[0]}/{plot_name}', data['trackID'].values[0])

# Save results
droplet_df = pd.DataFrame(drop_param)
droplet_df.to_csv('trajectory_characterization.csv', index=False)
print("Processing complete!")

Processing complete!
